In [ ]:
%%capture
!git clone https://github.com/huggingface/transformers.git
!pip install ./transformers
!pip install -U nlp

In [ ]:
%%capture
!pip install datasets

In [ ]:
%%capture
!pip install sentencepiece

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import datasets
import torch
import re
import numpy as np
import os

from tqdm import tqdm
from datasets import load_dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, MaxLengthCriteria

In [ ]:
%%capture
gsm8k = load_dataset('gsm8k', 'main')

In [ ]:
gsm8k_test = gsm8k['test']

validation_index = np.load("/content/drive/MyDrive/Thesis BAI/PromptTuning/validation_index.npy")
validation_data = gsm8k['train'].select(validation_index)

In [ ]:
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-large")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=True`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-large")
model = model.to("cuda:0")

In [ ]:
q_lens = [len(d['question']) for d in gsm8k['train']]

In [ ]:
np.percentile(q_lens, [50, 80, 90, 95, 99, 100])

array([217., 300., 361., 414., 527., 985.])

In [ ]:
def test_answer(pred_str, ans_str):
    pattern = '\d*\.?\d+'
    pred = re.findall(pattern, pred_str)
    if(len(pred) >= 1):
        # print(pred_str)
        pred = pred[-1]
        gold = re.findall(pattern, ans_str)
        # print(ans_str)
        gold = gold[-1]
        return pred == gold
    else: return False

def parse_pred_ans(filename):
    with open(filename, encoding='utf-8') as fd: lines = fd.readlines()
    am, a = None, None
    q = ''
    num_q, acc = 0, 0
    current_mode = 'none'
    questions = []
    ans_pred = []
    ans_gold = []
    for l in lines:
        if(l.startswith('Q: ')):
            if(am is not None and a is not None):
                questions.append(q)
                ans_pred.append(am)
                ans_gold.append(a)
                if(test_answer(am, a)):
                    acc += 1
            current_mode = 'q'
            q = l
            num_q += 1
        elif(l.startswith('A_model:')):
            current_mode = 'am'
            am = l
        elif(l.startswith('A:')):
            current_mode = 'a'
            a = l
        else:
            if(current_mode == 'q'): q += l
            elif(current_mode == 'am'): am += l
            elif(current_mode == 'a'): a += l
            else:
                raise ValueError(current_mode)

    questions.append(q)
    ans_pred.append(am)
    ans_gold.append(a)
    if(test_answer(am, a)):
        acc += 1
    print('num_q %d correct %d ratio %.4f' % (num_q, acc, float(acc / num_q)))
    return questions, ans_pred, ans_gold

#Prompt0

In [ ]:
#Prompt0
prompt_direct = open('/content/drive/MyDrive/Thesis BAI/PromptTuning/prompt_direct.txt').read()

In [ ]:
with open('/content/drive/MyDrive/Thesis BAI/PromptTuning/dev_flan_t5large_direct.txt', 'w', encoding='utf-8') as fd:
    for q, a in tqdm(zip(validation_data['question'], validation_data['answer']), total=len(validation_data['question'])):
        prompt_q = prompt_direct + '\nQuestion: ' + q + '\n'
        input_ids = tokenizer(prompt_q, return_tensors="pt").input_ids
        input_ids = input_ids.to("cuda:0")
        outputs = model.generate(input_ids, max_length=256)
        ans_ = tokenizer.decode(outputs[0])
        fd.write('Q: %s\nA_model:\n%s\nA:\n%s\n\n' % (q, ans_, a))

100%|██████████| 200/200 [06:37<00:00,  1.99s/it]


In [ ]:
_, _, _ = parse_pred_ans('/content/drive/MyDrive/Thesis BAI/PromptTuning/dev_flan_t5large_direct.txt')

num_q 200 correct 20 ratio 0.1000


#Prompt1

In [ ]:
#Prompt1
prompt_original = open('/content/drive/MyDrive/Thesis BAI/PromptTuning/prompt_original.txt').read()

In [ ]:
with open('/content/drive/MyDrive/Thesis BAI/PromptTuning/dev_flan_t5large_original.txt', 'w', encoding='utf-8') as fd:
    for q, a in tqdm(zip(validation_data['question'], validation_data['answer']), total=len(validation_data['question'])):
        prompt_q = prompt_original + '\nQuestion: ' + q + '\n'
        input_ids = tokenizer(prompt_q, return_tensors="pt").input_ids
        input_ids = input_ids.to("cuda:0")
        outputs = model.generate(input_ids, max_length=256)
        ans_ = tokenizer.decode(outputs[0])
        fd.write('Q: %s\nA_model:\n%s\nA:\n%s\n\n' % (q, ans_, a))

100%|██████████| 200/200 [10:02<00:00,  3.01s/it]


In [ ]:
_, _, _ = parse_pred_ans('/content/drive/MyDrive/Thesis BAI/PromptTuning/dev_flan_t5large_original.txt')

num_q 200 correct 19 ratio 0.0950


#Prompt2

In [ ]:
#Prompt2
prompt_easy = open('/content/drive/MyDrive/Thesis BAI/PromptTuning/prompt_easy.txt', encoding='utf-8').read()

In [ ]:
with open('/content/drive/MyDrive/Thesis BAI/PromptTuning/dev_flan_t5large_easy.txt', 'w', encoding='utf-8') as fd:
    for q, a in tqdm(zip(validation_data['question'], validation_data['answer']), total=len(validation_data['question'])):
        prompt_q = prompt_easy + '\nQuestion: ' + q + '\n'
        input_ids_easy = tokenizer(prompt_q, return_tensors="pt").input_ids
        input_ids_easy = input_ids_easy.to("cuda:0")
        outputs_easy = model.generate(input_ids_easy, max_length=256)
        ans_ = tokenizer.decode(outputs_easy[0])
        fd.write('Q: %s\nA_model:\n%s\nA:\n%s\n\n' % (q, ans_, a))

100%|██████████| 200/200 [10:01<00:00,  3.01s/it]


In [ ]:
_, _, _ = parse_pred_ans('/content/drive/MyDrive/Thesis BAI/PromptTuning/dev_flan_t5large_easy.txt')

num_q 200 correct 18 ratio 0.0900


#Prompt3

In [ ]:
#Prompt3
prompt_random = open('/content/drive/MyDrive/Thesis BAI/PromptTuning/prompt_random.txt', encoding="utf8").read()

In [ ]:
with open('/content/drive/MyDrive/Thesis BAI/PromptTuning/dev_flan_t5large_random.txt', 'w', encoding='utf-8') as fd:
    for q, a in tqdm(zip(validation_data['question'], validation_data['answer']), total=len(validation_data['question'])):
        prompt_q = prompt_random + '\nQuestion: ' + q + '\n'
        input_ids_rand = tokenizer(prompt_q, return_tensors="pt").input_ids
        input_ids_rand = input_ids_rand.to("cuda:0")
        outputs_rand = model.generate(input_ids_rand, max_length=256)
        ans_ = tokenizer.decode(outputs_rand[0])
        fd.write('Q: %s\nA_model:\n%s\nA:\n%s\n\n' % (q, ans_, a))

100%|██████████| 200/200 [10:16<00:00,  3.08s/it]


In [ ]:
_, _, _ = parse_pred_ans('/content/drive/MyDrive/Thesis BAI/PromptTuning/dev_flan_t5large_random.txt')

num_q 200 correct 17 ratio 0.0850


#Prompt4

In [ ]:
#Prompt4
prompt_hard = open('/content/drive/MyDrive/Thesis BAI/PromptTuning/prompt_hard.txt', encoding='utf-8').read()

In [ ]:
with open('/content/drive/MyDrive/Thesis BAI/PromptTuning/dev_flan_t5large_hard.txt', 'w', encoding='utf-8') as fd:
    for q, a in tqdm(zip(validation_data['question'], validation_data['answer']), total=len(validation_data['question'])):
        prompt_q = prompt_hard + '\nQuestion: ' + q + '\n'
        input_ids_hard = tokenizer(prompt_q, return_tensors="pt").input_ids
        input_ids_hard = input_ids_hard.to("cuda:0")
        outputs_hard = model.generate(input_ids_hard, max_length=256)
        ans_ = tokenizer.decode(outputs_hard[0])
        fd.write('Q: %s\nA_model:\n%s\nA:\n%s\n\n' % (q, ans_, a))

100%|██████████| 200/200 [11:03<00:00,  3.32s/it]


In [ ]:
_, _, _ = parse_pred_ans('/content/drive/MyDrive/Thesis BAI/PromptTuning/dev_flan_t5large_hard.txt')

num_q 200 correct 16 ratio 0.0800
